In [ ]:
%%bash
pip install astroML
pip install sklearn

In [ ]:
import numpy as np
from jointpdf.jointpdf import JointProbabilityMatrix
from jointpdf.jointpdf import FullNestedArrayOfProbabilities

In [ ]:
from functools import reduce

def compute_joint(multi_index, marginal_distributions):
    return reduce(lambda x, y: x*y, [marginal_distributions[combination] for
            combination in zip(range(len(multi_index)), multi_index)])

def compute_joint_from_marginals(marginal_distributions):
    dimension_joint_distribution = [len(dist) for dist in marginal_distributions]
    full_distribution_temp = np.zeros(dimension_joint_distribution)
    full_distribution = np.zeros(dimension_joint_distribution)

    it = np.nditer(full_distribution, flags=['multi_index'], op_flags=[['readwrite']])
    while not it.finished:
        full_distribution[it.multi_index]= compute_joint(it.multi_index, marginal_distributions)
        it.iternext()

    return full_distribution

marginal_distributions2 = np.array([[0.3, 0.7], [0.3, 0.7]])
print(compute_joint_from_marginals(marginal_distributions2))

def xor(it):
    """apply xor to binary numbers x1 and x2"""
    return 0 if it[0]==it[1] else 1

def double_xor(it):
    """apply xor to every pair of arguments in xor and return the outcome
    
    params:
        it: 1-d iterable with binary entries and len(it)%2==0
    """

    return [xor(it[2*i:2*i+2]) for i in range(len(it)/2)]
        
print(double_xor((1,1,0,1)))



In [ ]:
class JointProbabilityMatrixExtended(JointProbabilityMatrix):
    """this class extends the JointProbabilityMatrix by allowing different
    variables to have different statespaces"""
    def __init__(self, numvariables, max_num_states, joint_pdf):
        self.numvariables = numvariables
        self.max_num_states = max_num_states
        self.joint_probabilities = joint_pdf 
        #super(JointProbabilityMatrix, self).__init__(num_variables, num_states, joint_pdf)
    
    #needed to access methods in parent class
    def adjust_to_old_format(self):
        """ensure that every variable in joint has as many states and add states as needed"""
        self.old_joint_probabilities = self.joint_probabilities
        max_number_of_states = max(self.joint_probabilities.joint_probabilities.shape)
        temp_joint_probabilities = np.zeros([max_number_of_states]*self.numvariables)
        it = np.nditer(self.joint_probabilities.joint_probabilities, flags=['multi_index'])
        while not it.finished:
            temp_joint_probabilities[it.multi_index] = it.value
            it.iternext()
            
        self.joint_probabilities.joint_probabilities = temp_joint_probabilities
        self.numvalues = max_number_of_states
    
    #rewrite this to work on a nd-array instead!
    def append_determinstic_function(self, func, num_variables, num_states):
        """append an extra variable to the distribution given by the 
        deterministic function func
        
        params:
            func: a deterministic discrete function over the already defined variables
            num_variables: the number of variables
            num_states: iterable, the number of outcomes of the function
        """
        old_shape = list(self.joint_probabilities.joint_probabilities.shape)
        new_shape = old_shape + num_states
        dummy_joint_probability = np.zeros(new_shape)
        temp2_joint_probability = np.zeros(new_shape)
            
        it = np.nditer(dummy_joint_probability, flags=['multi_index'])
        while not it.finished:
            arguments = tuple(list(it.multi_index)[:-num_variables])
            if np.all(np.array(func(arguments)) == np.array(it.multi_index[-num_variables:])):
                temp2_joint_probability[it.multi_index] = self.joint_probabilities.joint_probabilities[arguments]
                
            it.iternext()
            
        self.joint_probabilities.joint_probabilities = temp2_joint_probability
        self.numvariables += num_variables 
                
    

In [ ]:
marginal_distributions = np.array([[0.3, 0.7], [0.4, 0.6], [0.5, 0.5], [0.8,  0.2]])



pdf = JointProbabilityMatrix(4, 2, compute_joint_from_marginals(marginal_distributions))
pdf_extended = JointProbabilityMatrixExtended(
    4, 2, FullNestedArrayOfProbabilities(compute_joint_from_marginals(marginal_distributions))
)
pdf_extended.adjust_to_old_format()

marginal_extended = pdf_extended.marginalize_distribution([0, 1])
marginal = pdf.marginalize_distribution([0, 1])

pdf_extended.append_determinstic_function(double_xor, num_variables=2, num_states=[2, 2])
pdf_extended.adjust_to_old_format()
marginal_extended = pdf_extended.marginalize_distribution([0, 1])

print(
    np.all(
        marginal_extended.joint_probabilities.joint_probabilities==marginal.joint_probabilities.joint_probabilities
    )
)


In [ ]:
import itertools

def create_mi_profile(joint_distribution, function_labels):
    """create a mutual information profile
    
    params:
        joint_distribution: A JointProbabilityMatrix object from the jointpdf package.
        The joint probabilities properties should be of both all the variables and the
        function.
        function_labels: The labels in the joint distribution which the function represents
        
    returns: a 1-d nd-array where every value represents the average normalized mutual information
        between subsets of variables (of the size of the index) and the ouput variable. 
    """

    number_of_arguments = joint_distribution.numvariables-len(function_labels)
    mi_values = np.zeros(number_of_arguments+1)
    for number_of_variables in np.arange(1, number_of_arguments+1, 1):
        combinations = itertools.combinations(range(number_of_arguments), 
                                              number_of_variables)
        
        mi_values[number_of_variables] = np.mean(
            [joint_distribution.mutual_information(combination, function_labels) 
             for combination in combinations]
        )
        for comb in itertools.combinations(range(number_of_arguments), number_of_variables):
            print(comb)
        
    return mi_values
                
    

In [ ]:
pdf_extended = JointProbabilityMatrixExtended(
    2, 2, FullNestedArrayOfProbabilities(compute_joint_from_marginals(marginal_distributions[:2]))
)
pdf_extended.adjust_to_old_format()

#print(pdf_extended.joint_probabilities.joint_probabilities)
#print("check mutual info {}".format(pdf_extended.mutual_information([0], [1])))

pdf_extended.append_determinstic_function(xor, num_variables=1, num_states=[2])
pdf_extended.adjust_to_old_format()

print(pdf_extended.joint_probabilities.joint_probabilities)
print("second check mutual information {}".format(pdf_extended.mutual_information([0], [2])))

calulate_mutual_information(
    pdf_extended.marginalize_distribution([0]).joint_probabilities.joint_probabilities,
    pdf_extended.marginalize_distribution([2]).joint_probabilities.joint_probabilities,
    pdf_extended.marginalize_distribution([0,2]).joint_probabilities.joint_probabilities
)

create_mi_profile(pdf_extended, [2])

In [ ]:
def calculate_entropy(distribution_values):
    """ calculate the entropy
    
    params:
        distribution_values should be a 1-dim numpy array, summing to 1
    
    returns: the entropy value
    """
    dist_values = distribution_values[distribution_values != 0]
    return - np.sum(np.log2(dist_values) * dist_values)

def calulate_mutual_information(distribution1, distribution2, joint_distribution):
    return (calculate_entropy(distribution1) +
            calculate_entropy(distribution2) -
            calculate_entropy(joint_distribution))